<a href="https://colab.research.google.com/github/AUT-Student/BigData-Project/blob/main/BigData_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [1]:
from dataclasses import dataclass
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import math
import glob

# Load Datasets

## CAIDA

In [2]:
!gdown https://snap.stanford.edu/data/as-caida.tar.gz
!mkdir caida
!tar -C /content/caida -xzf /content/as-caida.tar.gz 

Downloading...
From: https://snap.stanford.edu/data/as-caida.tar.gz
To: /content/as-caida.tar.gz
100% 46.4M/46.4M [00:01<00:00, 28.6MB/s]


In [3]:
!grep as-caida2006 /content/caida/* | wc -l

52


In [4]:
!grep as-caida2007 /content/caida/* | wc -l

46


In [5]:
!head /content/caida/as-caida20060102.txt -n 40

# Directed graph: as-caida20060102.txt
# The CAIDA AS Relationships Dataset, from 01 02 2006
# Relationships:	-1 (<FromNodeId> is a customer of <ToNodeId>)
# 			1 (<FromNodeId> is a provider of <ToNodeId>)
# 			0 (<FromNodeId> and <ToNodeId> are peers)
# 			2 (<FromNodeId> and <ToNodeId> are siblings (the same organization).)
# Nodes:21202	Edges: 85850
# FromNodeId        ToNodeId	Relationship
8563	26716	1
8563	2914	-1
8563	21414	-1
8563	21482	1
8563	3277	0
8563	4323	-1
28801	702	-1
28801	8422	-1
28801	8220	-1
5006	20283	1
5006	21902	1
5006	25637	1
5006	17162	1
5006	18581	1
5006	11401	1
5006	16904	1
5006	21730	1
5006	32461	1
5006	7018	-1
5006	14278	1
5006	1347	2
5006	32664	1
5006	33111	1
5006	26021	1
5006	23359	1
5006	21951	1
5006	7792	1
5006	16595	1
5006	10337	1
5006	18699	1
5006	32110	1
5006	7973	1


In [6]:
!head /content/caida/as-caida20040105.txt -n 40

# Directed graph: as-caida20040105.txt
# The CAIDA AS Relationships Dataset, from 01 05 2004
# Relationships:	-1 (<FromNodeId> is a customer of <ToNodeId>)
# 			1 (<FromNodeId> is a provider of <ToNodeId>)
# 			0 (<FromNodeId> and <ToNodeId> are peers)
# 			2 (<FromNodeId> and <ToNodeId> are siblings (the same organization).)
# Nodes:16301	Edges: 65910
# FromNodeId        ToNodeId	Relationship
6712	1668	0
6712	3215	-1
8563	26716	1
8563	4323	-1
13666	19262	-1
28689	20804	-1
28689	6778	-1
28801	702	0
28801	8422	-1
5006	20283	1
5006	21902	1
5006	25637	1
5006	17162	1
5006	18581	1
5006	11401	1
5006	16904	1
5006	21730	1
5006	7018	-1
5006	14278	1
5006	18454	1
5006	1347	1
5006	26021	1
5006	7792	1
5006	16595	1
5006	10337	1
5006	4045	1
5006	18699	1
5006	7973	1
5006	25615	1
5006	18723	1
5006	1239	-1
5006	10765	1


## Oregon

In [7]:
!rm oregon -r

rm: cannot remove 'oregon': No such file or directory


In [8]:
!mkdir oregon
%cd oregon

!gdown https://snap.stanford.edu/data/oregon1_010331.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010407.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010414.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010421.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010428.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010505.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010512.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010519.txt.gz
!gdown https://snap.stanford.edu/data/oregon1_010526.txt.gz


!gunzip oregon1_010331.txt.gz
!gunzip oregon1_010407.txt.gz
!gunzip oregon1_010414.txt.gz
!gunzip oregon1_010421.txt.gz
!gunzip oregon1_010428.txt.gz
!gunzip oregon1_010505.txt.gz
!gunzip oregon1_010512.txt.gz
!gunzip oregon1_010519.txt.gz
!gunzip oregon1_010526.txt.gz

%cd ..

/content/oregon
Downloading...
From: https://snap.stanford.edu/data/oregon1_010331.txt.gz
To: /content/oregon/oregon1_010331.txt.gz
100% 69.1k/69.1k [00:00<00:00, 1.15MB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010407.txt.gz
To: /content/oregon/oregon1_010407.txt.gz
100% 69.3k/69.3k [00:00<00:00, 1.45MB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010414.txt.gz
To: /content/oregon/oregon1_010414.txt.gz
100% 69.8k/69.8k [00:00<00:00, 1.55MB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010421.txt.gz
To: /content/oregon/oregon1_010421.txt.gz
100% 70.7k/70.7k [00:00<00:00, 1.17MB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010428.txt.gz
To: /content/oregon/oregon1_010428.txt.gz
100% 70.1k/70.1k [00:00<00:00, 1.17MB/s]
Downloading...
From: https://snap.stanford.edu/data/oregon1_010505.txt.gz
To: /content/oregon/oregon1_010505.txt.gz
100% 70.5k/70.5k [00:00<00:00, 1.41MB/s]
Downloading...
From: https://snap.stanford

In [9]:
!head /content/oregon/oregon1_010331.txt

# Undirected graph: oregon1_010331.txt
# AS peering information inferred from Oregon route-views BGP data, from March 31 2001
# Nodes: 10670 Edges: 22002
# FromNodeId	ToNodeId
10000	4725
4725	6805
4725	7523
4725	7524
4725	7673
4725	9352


In [11]:
!grep "13237" /content/caida/as-caida20040105.txt | wc -l

862


# Data Structure Classes

## Edge Class

In [12]:
@dataclass(init=True, eq=True, frozen=True)
class Edge:
  source: int
  destination: int

  def get_reverse(self):
    return Edge(source=self.destination, destination=self.source)

  def get_minmax(self):
    return Edge(source=min(self.source, self.destination),
                destination=max(self.source, self.destination))

## Graph Class

In [13]:
class Graph():
  def __init__(self, directed):
    self.edges = set()
    self.nodes = set()
    self.node_order = list()
    self.node_order_set = set()

    self.reset_statics()

  def _set_order(self, node):
    if node not in self.node_order_set:
      self.node_order_set.add(node)
      self.node_order.append(node)

  def add_edge(self, edge:Edge):
    self.nodes.add(edge.source)
    self.nodes.add(edge.destination)
    self.edges.add(edge)

    self._set_order(edge.source)

  def finish(self):
    for node in self.nodes:
      # Set order for remaining nodes
      self._set_order(node)

    self.reset_statics()

  def reset_statics(self):
    self.number_trees = None
    self.node_adjacency_list = None

  def number_nodes(self):
    return len(self.nodes)
  
  def number_edges(self):
    return len(self.edges)

  def get_edges(self):
    return self.edges
  
  def get_nodes(self):
    return self.nodes

  def get_node(self, node_id):
    return self.node_order[node_id]
  
  def get_node_adjacency_list(self, node_id):
    if self.node_adjacency_list is None:
      self.calculate_node_adjacency_list()
    
    return self.node_adjacency_list[self.get_node(node_id)]

  def calculate_node_adjacency_list(self):
    # if self.node_adjacency_list is not None:
    #   return

    self.node_adjacency_list = dict()

    for node in self.nodes:
      self.node_adjacency_list[node] = set()
    
    for edge in self.edges:
      self.node_adjacency_list[edge.source].add(edge.destination) 
      self.node_adjacency_list[edge.destination].add(edge.source) 
    
  def calculate_number_trees(self):
    if self.number_trees is not None:
      return self.number_trees

    # if self.node_adjacency_list is None:
    #   self.calculate_node_adjacency_list()

    self.number_trees = 0
    for node_x in self.nodes:
      for node_y in self.node_adjacency_list[node_x]:
        for node_z in self.node_adjacency_list[node_x]:

          if node_x < node_y < node_z:
            if node_z in self.node_adjacency_list[node_y]:
              self.number_trees += 1

  def get_number_trees(self):
    if self.number_trees is None:
      self.calculate_number_trees()

    return self.number_trees

  def get_number_trees_per_edge(self, edge):
    # if self.node_adjacency_list is None:
    #   self.calculate_node_adjacency_list()

    node_x = edge.source
    node_y = edge.destination

    number_trees = 0

    for node_z in self.node_adjacency_list[node_x]:
      if node_z in self.node_adjacency_list[node_y]:
        number_trees += 1
    
    return number_trees

## Snapshot Graphs

In [14]:
class Snapshot():
  def __init__(self, dataset_path):
    self.dataset_path = dataset_path
    self.graphs = list()

  def load_graphs(self):
    for path in sorted(glob.glob(pathname=self.dataset_path, recursive=True)):
      print(".", end="")
      graph = Graph(directed=False)

      with open(path, "r") as file:
        for i, line in enumerate(file.readlines()):
          if line[0] == "#": continue
          
          source, destination = line[:-1].split("\t")[:2]
          if source == destination: continue
          edge = Edge(source=int(source), destination=int(destination))
          graph.add_edge(edge)
        
      graph.finish()
      graph.calculate_node_adjacency_list()
      self.graphs.append(graph)

  def get_graph(self, i):
    return self.graphs[i]
  
  def number_graphs(self):
    return len(self.graphs)

# Load Dataset

## Oregon

In [15]:
oregon = Snapshot(dataset_path = "/content/oregon/*")

In [16]:
oregon.load_graphs()

.........

In [17]:
oregon.number_graphs()

9

## Caida 2006

In [18]:
caida2006 = Snapshot(dataset_path = "/content/caida/as-caida2006*")

In [19]:
caida2006.load_graphs()

....................................................

In [20]:
caida2006.number_graphs()

52

## Caida 2007

In [21]:
caida2007 = Snapshot(dataset_path = "/content/caida/as-caida2007*")

In [22]:
caida2007.load_graphs()

..............................................

In [23]:
caida2007.number_graphs()

46

# Algorithm 1

## Oracle

In [24]:
class Oracle():
  def __init__(self, graph:Graph, rho=None):
    self.graph = graph
    self.rho = rho
    self.T = None
    self.N_edge = None

  def set_rho(self, rho):
    self.rho = rho

  def train(self):
    print("Cacluate total trees...")
    self.graph.calculate_number_trees()
    self.T = self.graph.get_number_trees()
    self._calculate_number_trees_per_edge()
    print("Training oracle completed!")

  def _calculate_number_trees_per_edge(self):
    N_edge_complete = list()

    print("Counting Trees per edges...")
    edges = self.graph.get_edges()
    for i, edge in enumerate(edges):
      if True and i % (len(edges)//10)==0: print(f"{round(i*100/len(edges), 0)}%", end=" ")
      if edge.source < edge.destination:
        number_trees = self.graph.get_number_trees_per_edge(edge)
        
        N_edge_complete.append({"edge": edge, "number_trees": number_trees})

    print("\nSorting and selecting top N_e")
    N_edge_top = sorted(N_edge_complete, key=lambda x:-x["number_trees"])[:len(N_edge_complete)//10]

    self.N_edge = dict()
    for item in N_edge_top:
      self.N_edge[item["edge"]] = item["number_trees"]

  def is_heavy(self, edge):
    edge = edge.get_minmax()
    if edge in self.N_edge:
      return self.N_edge[edge] >= self.T/self.rho
    else:
      return False

## Estimator

In [25]:
class Estimator():
  def __init__(self, graph:Graph, oracle:Oracle, epsilon,
               alpha, beta, gamma):
    self.graph = graph 
    self.oracle = oracle

    self.epsilon = epsilon
    self.alpha = alpha
    self.gamma = gamma
    self.beta = beta

    self.A_l, self.A_m, self.A_h = 0, 0, 0
    self.S_l, self.S_m, self.S_aux = set(), set(), set()

    self.T = self.oracle.T

    self.C = dict()
    self.B = dict()

    self.n = graph.number_nodes()
    self.m = graph.number_edges()

    self.set_probabilities()

  def set_probabilities(self):
    if self.T >= ((self.m/self.epsilon) ** 0.5):
      self.rho = (self.m * self.T) ** (1/3)
      self.p1 = self.alpha * (self.epsilon ** -2) / self.rho
      self.p2 = min(self.beta*(self.epsilon ** -2) * self.rho / self.T , 1)
      self.p3 = self.gamma * (self.epsilon ** (-2)) * math.log(self.n) / self.rho
    
    else:
      self.rho = (self.m ** 0.5) / self.epsilon
      self.p1 = 0
      self.p2 = 1
      self.p3 = self.gamma * (self.epsilon ** (-2)) / self.rho
    
    print(f"T = {self.T}, rho = {self.rho}, p1 = {self.p1}, p2 = {self.p2}, p3 = {self.p3}")
    self.oracle.set_rho(self.rho)

  def estimate_number_trees(self):
    for node_id in range(self.graph.number_nodes()):
      if node_id%(self.graph.number_nodes()//10)==0 : print(f"{round(node_id*100/self.graph.number_nodes())}%", end=" ")
      v = self.graph.get_node(node_id)
      N_v = self.graph.get_node_adjacency_list(node_id)

      for ab in self.S_l.union(self.S_m):
        if ab.source in N_v and ab.destination in N_v:
          self.C[ab] += 1
      
      for a in N_v:
        av = Edge(a, v)

        if av in self.S_aux:
          self.B[av] = 1

      for u in N_v:
        vu = Edge(v, u)
        uv = Edge(u, v)
        
        if random.random() < self.p3:
          self.S_aux.add(vu)
          self.B[vu] = 0

        if self.oracle.is_heavy(uv) is False:
          if uv in self.S_l:
            self.A_l += self.C[uv]
          elif random.random() < self.p1:
            self.S_l.add(vu)
            self.C[vu] = 0
        else:
          sharp = 0
          for edge in self.S_aux:
            if edge.source == u and self.B[edge]==1 and edge.destination in N_v:
              sharp += 1
          
          if sharp >= self.p3 * self.rho:
            self.A_h += sharp
          elif uv in self.S_m:
            self.A_m += self.C[uv]
          elif random.random() < self.p2:
            self.S_m.add(vu)
            self.C[vu] = 0
    
    print(f"Len(S_l) = {len(self.S_l)}, Len(S_m) = {len(self.S_m)}, Len(S_aux)={len(self.S_aux)}")
    print(f"A_l {self.A_l}, A_m = {self.A_m}, A_h = {self.A_h}")
    return self.A_l / self.p1 + self.A_m / self.p2 + self.A_h / self.p3

# Results

## Oregon

In [26]:
oregon.get_graph(1).number_edges()

22000

In [27]:
oracle_oregon = Oracle(graph=oregon.get_graph(0))
oracle_oregon.train()

Cacluate total trees...
Counting Trees per edges...
0.0% 10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% 100.0% 
Sorting and selecting top N_e
Training oracle completed!


In [28]:
estimator_oregon = Estimator(graph=oregon.get_graph(1), oracle=oracle_oregon
                             , epsilon=0.05, alpha = 1, beta = 1, gamma = 0.1)

T = 17144, rho = 722.5118032258747, p1 = 0.5536241736315971, p2 = 1, p3 = 0.5138022987693875


In [29]:
estimated_number_trees_oregon = estimator_oregon.estimate_number_trees()

0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% Len(S_l) = 17437, Len(S_m) = 272, Len(S_aux)=22521
A_l 7674, A_m = 2078, A_h = 0


In [30]:
oregon.get_graph(1).get_number_trees()

15834

In [31]:
estimated_number_trees_oregon

15939.388944888407

## Caida 2006

In [32]:
caida2006.get_graph(1).number_nodes()

21157

In [33]:
caida2006.get_graph(1).number_edges()

85468

In [34]:
oracle_caida2006 = Oracle(graph=caida2006.get_graph(0))
oracle_caida2006.train()

Cacluate total trees...
Counting Trees per edges...
0.0% 10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% 
Sorting and selecting top N_e
Training oracle completed!


In [35]:
estimator_caida2006 = Estimator(graph=caida2006.get_graph(1), oracle=oracle_caida2006
                               , epsilon=0.05, alpha = 3, beta = 1, gamma = 0.3)

T = 30433, rho = 1375.253532356451, p1 = 0.8725663826827915, p2 = 1, p3 = 0.8690522174701264


In [36]:
estimated_number_trees_caida2006 = estimator_caida2006.estimate_number_trees()

0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% Len(S_l) = 41440, Len(S_m) = 568, Len(S_aux)=74325
A_l 17309, A_m = 10808, A_h = 0


In [37]:
estimated_number_trees_caida2006

30644.88615963151

In [38]:
caida2006.get_graph(1).get_number_trees()

30689